In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())

print(emission_df.head())
results_temp = session.query(Temp_change)

#print(results) 

temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']

        Entity Code  Year  AnnualCO2emissions
0  Afghanistan  AFG  1961              490798
1  Afghanistan  AFG  1962              688594
2  Afghanistan  AFG  1963              706736
3  Afghanistan  AFG  1964              838551
4  Afghanistan  AFG  1965             1006917


In [7]:




season_df = temp_df.loc[temp_df["Months"].isin(selection)]

#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
# Calculate avg temp per Meteorological year
meteor_df['avg_temp']= meteor_df.mean(axis =1)
meteor_id_df = meteor_df.set_index('Area')

session.close() 

C:\Users\ubc\anaconda3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
season_df



,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
12,24,Afghanistan,DecJanFeb,Temperature change,°C,-0.731,1.042,1.981,-2.012,-0.621,...,1.921,0.449,-1.858,0.753,-0.549,1.549,1.905,1.436,1.714,1.449
13,26,Afghanistan,MarAprMay,Temperature change,°C,0.073,0.454,0.211,0.322,-0.972,...,2.467,2.034,0.600,1.132,0.489,0.985,2.174,1.789,2.238,0.593
14,28,Afghanistan,JunJulAug,Temperature change,°C,0.406,-0.330,0.535,0.003,-0.331,...,0.558,1.637,0.878,1.312,0.956,0.990,0.992,1.002,1.528,0.790
15,30,Afghanistan,SepOctNov,Temperature change,°C,-0.066,-1.645,0.800,-1.222,1.123,...,1.311,1.249,0.788,1.751,1.000,0.859,0.821,1.704,0.614,0.739
29,58,Albania,DecJanFeb,Temperature change,°C,0.638,-0.344,-1.474,-0.789,-1.078,...,1.306,0.606,-1.005,0.716,2.350,0.856,2.217,-0.240,1.182,0.262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,9618,Non-Annex I countries,SepOctNov,Temperature change,°C,-0.071,0.075,0.245,-0.402,0.062,...,1.144,0.775,1.028,0.910,1.041,1.450,1.204,1.216,1.155,1.486
3375,9646,OECD,DecJanFeb,Temperature change,°C,0.561,-0.362,-0.222,0.098,-0.732,...,1.272,0.770,1.709,1.327,0.686,1.307,2.636,1.884,1.453,1.527
3376,9648,OECD,MarAprMay,Temperature change,°C,-0.038,-0.189,0.141,-0.590,-0.110,...,1.742,0.390,1.442,0.475,0.778,1.191,1.928,1.237,1.241,1.352
3377,9650,OECD,JunJulAug,Temperature change,°C,0.101,0.052,-0.047,-0.114,-0.495,...,1.012,0.954,1.061,1.119,0.779,0.958,1.303,1.081,1.078,1.078


In [9]:
country = 'Grenada'
season_country_mean = season_df.loc[season_df['Area'] == country]
season_country_mean

,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1060,3152,Grenada,DecJanFeb,Temperature change,°C,0.292,-0.097,0.133,0.286,-0.722,...,1.257,1.084,0.878,1.349,0.887,1.076,1.147,0.884,0.800,0.616
1061,3154,Grenada,MarAprMay,Temperature change,°C,0.397,0.418,-0.115,-0.045,-0.268,...,1.687,0.510,0.442,0.977,0.744,0.790,1.091,1.031,0.426,0.837
1062,3156,Grenada,JunJulAug,Temperature change,°C,0.172,-0.227,0.272,-0.541,-0.341,...,1.487,1.316,0.898,1.055,1.210,1.197,1.340,1.298,0.853,1.256
1063,3158,Grenada,SepOctNov,Temperature change,°C,-0.157,0.069,0.448,-0.332,0.035,...,1.280,0.923,1.309,0.855,0.871,1.303,0.902,1.155,0.863,1.169


In [10]:
season_country_mean = season_country_mean.drop(['field1', 'Element', 'Unit'], 1).reset_index(drop=True)
season_country_mean

,Area,Months,1961,1962,1963,1964,1965,1966,1967,1968,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Grenada,DecJanFeb,0.292,-0.097,0.133,0.286,-0.722,0.347,-0.077,-0.224,...,1.257,1.084,0.878,1.349,0.887,1.076,1.147,0.884,0.800,0.616
1,Grenada,MarAprMay,0.397,0.418,-0.115,-0.045,-0.268,-0.276,-0.809,-0.372,...,1.687,0.510,0.442,0.977,0.744,0.790,1.091,1.031,0.426,0.837
2,Grenada,JunJulAug,0.172,-0.227,0.272,-0.541,-0.341,-0.361,-0.411,-0.462,...,1.487,1.316,0.898,1.055,1.210,1.197,1.340,1.298,0.853,1.256
3,Grenada,SepOctNov,-0.157,0.069,0.448,-0.332,0.035,-0.112,-0.309,-0.219,...,1.280,0.923,1.309,0.855,0.871,1.303,0.902,1.155,0.863,1.169


In [11]:
year = season_country_mean.columns.drop(['Area', 'Months'])
year

Index(['1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [12]:
season_list = []
if len(season_country_mean) == 4:
    season_obj ={
        'Country': country,
        'Year': list(np.ravel(year)),
        'Winter':list(np.ravel(season_country_mean.iloc[0,2:].values)),
        'Spring':list(np.ravel(season_country_mean.iloc[1,2:].values)),
        'Summer':list(np.ravel(season_country_mean.iloc[2,2:].values)),
        'Fall':list(np.ravel(season_country_mean.iloc[3,2:].values)),
        'Data Found':'yes'
        }
elif len(season_country_mean) == 3 :
    print(f'...............\n Country: {country}')
    print(season_country_mean)
    season_obj ={
        'Country': country,
        'Year':list(np.ravel(year)),
        'Winter':list(np.ravel(season_country_mean.iloc[0,2:].values)),
        'Spring':list(np.ravel(season_country_mean.iloc[1,2:].values)),
        'Summer':list(np.ravel(season_country_mean.iloc[2,2:].values)),
        'Data Found':'yes'
        }
else:
    print(f'...............\n Country: {country}')
    print(season_country_mean)
    season_obj ={ 'Data Found' :'No' }
        
#Append the object to a list
season_list.append(season_obj)
    
    
    
season_list

[{'Country': 'Grenada',
  'Year': ['1961',
   '1962',
   '1963',
   '1964',
   '1965',
   '1966',
   '1967',
   '1968',
   '1969',
   '1970',
   '1971',
   '1972',
   '1973',
   '1974',
   '1975',
   '1976',
   '1977',
   '1978',
   '1979',
   '1980',
   '1981',
   '1982',
   '1983',
   '1984',
   '1985',
   '1986',
   '1987',
   '1988',
   '1989',
   '1990',
   '1991',
   '1992',
   '1993',
   '1994',
   '1995',
   '1996',
   '1997',
   '1998',
   '1999',
   '2000',
   '2001',
   '2002',
   '2003',
   '2004',
   '2005',
   '2006',
   '2007',
   '2008',
   '2009',
   '2010',
   '2011',
   '2012',
   '2013',
   '2014',
   '2015',
   '2016',
   '2017',
   '2018',
   '2019'],
  'Winter': [0.292,
   -0.097,
   0.133,
   0.286,
   -0.722,
   0.347,
   -0.077,
   -0.224,
   -0.064,
   0.541,
   -0.175,
   -0.5,
   0.073,
   -0.689,
   -0.615,
   -0.43,
   -0.11,
   0.097,
   0.266,
   0.359,
   0.598,
   0.465,
   0.14,
   -0.064,
   -0.467,
   -0.41,
   0.135,
   0.623,
   -0.213,
   0.088,

In [13]:
season_list

[{'Country': 'Grenada',
  'Year': ['1961',
   '1962',
   '1963',
   '1964',
   '1965',
   '1966',
   '1967',
   '1968',
   '1969',
   '1970',
   '1971',
   '1972',
   '1973',
   '1974',
   '1975',
   '1976',
   '1977',
   '1978',
   '1979',
   '1980',
   '1981',
   '1982',
   '1983',
   '1984',
   '1985',
   '1986',
   '1987',
   '1988',
   '1989',
   '1990',
   '1991',
   '1992',
   '1993',
   '1994',
   '1995',
   '1996',
   '1997',
   '1998',
   '1999',
   '2000',
   '2001',
   '2002',
   '2003',
   '2004',
   '2005',
   '2006',
   '2007',
   '2008',
   '2009',
   '2010',
   '2011',
   '2012',
   '2013',
   '2014',
   '2015',
   '2016',
   '2017',
   '2018',
   '2019'],
  'Winter': [0.292,
   -0.097,
   0.133,
   0.286,
   -0.722,
   0.347,
   -0.077,
   -0.224,
   -0.064,
   0.541,
   -0.175,
   -0.5,
   0.073,
   -0.689,
   -0.615,
   -0.43,
   -0.11,
   0.097,
   0.266,
   0.359,
   0.598,
   0.465,
   0.14,
   -0.064,
   -0.467,
   -0.41,
   0.135,
   0.623,
   -0.213,
   0.088,